# PDA

## Insert article into c then run all codes, results at the end

In [1]:
c = "1.5 million Singaporeans to get up to S$300 cash in special GST Voucher payment, all households to get S$100 utilities credit"

In [2]:
def trial(a):

    #if len > 60, use summerization
    links = google(a)  
    print(links)
    links = crawlall(links)
    crossref_results = test(a,links[0],links[1],links[2])

    
    return crossref_results

In [3]:
#using summarization to get keywords for search 
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
classifier = pipeline("sentiment-analysis")


def summarisation(c):
    b = summarizer(c, max_length=130, min_length=30, do_sample=False)
    #remove 'summary_text' heading
    b = str(b[17:])
    return(b)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [4]:
#pip install google
#crawl top 3 links from google search
def google(a):
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")

# to search
    query = a
    x = []
    for j in search(query, tld="co.in", num=1, stop=3, pause=2):
        x.append(j)
    #returning a list
    return(x)

In [5]:
#crawl part 1
#crawl content within the link
import requests
from bs4 import BeautifulSoup

def crawl(a):
    URL = a
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")

    company = soup.find_all('p')
    c = ""
    for i in range(0,len(company)):
        c = c + company[i].get_text()
    
    return(c)

In [6]:
#crawl all in the list part 2
def crawlall(a):
    xx = []
    for i in range(0,len(a)):
        y = str(crawl(a[i]))
        xx.append(y)
    return xx

In [7]:
#pip install -U sentence-transformers
#compare similarity between the content
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer, util

def test(a,b,c,d):
    sentences_one = [a,b]
    sentences_two = [a,c]
    sentences_three = [a,d]

    model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
    embeddings = model.encode(sentences_one)
    one = (util.pytorch_cos_sim(embeddings[0], embeddings[1]))
    print(one)

    embeddings = model.encode(sentences_two)
    two = (util.pytorch_cos_sim(embeddings[0], embeddings[1]))
    print(two)
    
    embeddings = model.encode(sentences_three)
    three = (util.pytorch_cos_sim(embeddings[0], embeddings[1]))
    print(three)
    
    avg = (one + two + three)/3
    print(avg)
    
    avg = avg.item()
    
    if avg >= 0.5:
        result = "real with" + " " + str(avg*100) + "% " + "confidence"
    
    else:
        result = "fake with" + " " + str((1-avg)*100) + "% " + "confidence"
    
    return(result)
    

In [8]:
trial(c)

['https://www.channelnewsasia.com/singapore/gst-voucher-cash-special-payment-singaporeans-household-utilities-credit-2759761', 'https://vulcanpost.com/792535/special-gst-cash-voucher-august-2022-singapore/', 'https://www.todayonline.com/singapore/15m-eligible-adult-sporeans-each-get-s300-cash-part-s15b-support-package-amid-rising-living-costs-1929501']
tensor([[0.7527]])
tensor([[0.7780]])
tensor([[0.7427]])
tensor([[0.7578]])


'real with 75.77913403511047% confidence'